In [1]:
import pandas as pd

# Wczytanie danych od Shumee
data = pd.read_excel("shumee_mckinsey -Aktualizacja 01.03.xlsx")

Na razie ograniczę się tylko do produktów sprzedanych w Polsce:

In [2]:
data = data[data["Kraj"]=="PL"]

Sprawdzam poprawność danych:

In [3]:
print("Liczba NaN w każdej kolumnie: ")
data.isna().agg("sum")

Liczba NaN w każdej kolumnie: 


ID zamówienia         0
Data                  0
Źródło              229
Kraj                  0
Miasto              270
Kod Pocztowy        311
Nazwa produktu        0
SKU               13175
EAN               14328
Ilość               112
Cena                112
Waluta              112
Koszt dostawy       112
Forma dostawy      3544
dtype: int64

Usunę te wiersze, w których nie ma ceny (jest to tylko 112 wierszy, więc nie tracę dużo danych usuwając je)

In [4]:
data = data[~data["Cena"].isna()]

Teraz załaduję model Word2Vec. Wykorzsytuję bibliotekę Gensim, oraz model wytrenowany m.in. na polskiej Wikipedii, który można pobrać pod tym adresem: https://github.com/sdadas/polish-nlp-resources

In [5]:
# !pip install --upgrade gensim

from gensim.models import KeyedVectors

word2vec = KeyedVectors.load("./word2vec/word2vec_100_3_polish.bin")

C:\Users\PiotrGrabysz\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Word2Vec zamienia słowa na wektory (w tym przypadku wektory wymiaru 100). Na przykład słowu "krzesło" odpowiada taki wektor:

In [13]:
word2vec["krzesło"]

array([ 2.012289e+00,  1.462072e+00, -5.004100e+00, -3.441711e+00,
        3.990560e-01, -2.565111e+00,  5.030282e+00,  6.047542e+00,
        4.548630e+00,  1.738084e+00, -2.758710e-01, -7.522800e-02,
        1.855085e+00, -6.247217e+00, -5.888261e+00, -8.400401e+00,
       -2.965125e+00, -3.482589e+00,  4.830710e-01, -1.018867e+00,
       -8.132490e-01, -3.048696e+00,  4.958340e+00, -1.293860e-01,
       -3.085849e+00, -1.863563e+00, -5.953629e+00,  4.385289e+00,
        1.279462e+00, -1.170168e+00, -8.543170e-01, -4.681480e-01,
        5.515535e+00, -7.276000e-03,  7.161128e+00,  3.059943e+00,
        2.034154e+00,  3.861800e-01,  2.643000e+00, -7.206631e+00,
       -2.734934e+00,  4.014585e+00,  3.858501e+00, -5.677879e+00,
        4.270895e+00,  6.786670e-01,  3.034365e+00,  1.110994e+00,
       -4.248924e+00,  2.391838e+00,  2.946400e-02,  1.036666e+00,
        5.377800e-01, -5.942716e+00, -4.273711e+00, -1.379860e+00,
        2.706288e+00,  2.586019e+00,  3.046250e+00,  1.442850e

Na podstawie takiej wektorowej reprezentacji można wyszukać słowa o najbardziej podobnym znaczeniu do danego słowa.

Słowa podobne do "krzesło":

In [16]:
print(word2vec.similar_by_word("krzesło"))

[('stołek', 0.9247931241989136), ('taboret', 0.9213142991065979), ('fotel', 0.9031204581260681), ('kanapa', 0.8804746270179749), ('krzesełko', 0.8769845962524414), ('sofa', 0.8673840165138245), ('zydel', 0.8485152125358582), ('tapczan', 0.8382896780967712), ('otomana', 0.8218340277671814), ('szezlong', 0.8153223991394043)]


Oraz słowa podobne do słowa "komputer"

In [17]:
print(word2vec.similar_by_word("komputer"))

[('oprogramowanie', 0.799543559551239), ('sterownik', 0.794553279876709), ('kalkulator', 0.7934095859527588), ('skaner', 0.7867334485054016), ('urządzenie', 0.7822620868682861), ('laptop', 0.7730702757835388), ('drukarka', 0.7705292105674744), ('procesor', 0.7685826420783997), ('czytnik', 0.7645211219787598), ('serwer', 0.7605681419372559)]


In [20]:
products_names = data["Nazwa produktu"].to_list()

In [21]:
products_names = [
    [word for word in name.lower().split()]
    for name in data["Nazwa produktu"].to_list()
]

Tworzę zbiór wszystkich słów wykorzystanych w nazwach produktów. Wszystkie wielkie litery zastępuję małymi literami, bo tylko takie słowa są rozpoznawane przez word2vec.

In [6]:
words_set = set()

stoplist = []
for name in data["Nazwa produktu"].to_list():
    for word in name.lower().split():
        words_set.add(word)

In [7]:
len(words_set)

33708

W nazwach produktów użyto 33708 różnych słów. Być może niektóre z nich, np oznaczające wymiary (jak 10x10), trzeba będzie odrzucić.

In [8]:
# Słowa które znalazły się w banku słów word2vec:
included_words = []
# Słowa których word2vec nie zna
removed_words = []
# lista wektorów odpowiadających poszeczególnym słowom
words_array = []

for word in words_set:
    try:
        vec = word2vec[word]
        words_array.append(vec)
        included_words.append(word)
    except KeyError:
        removed_words.append(word)

In [9]:
print(f"Udało się zakodować {len(included_words)}")
print(f"{len(removed_words)} słów nie znalazło się w bazie słów")

Udało się zakodować 9190
24518 słów nie znalazło się w bazie słów


In [10]:
import numpy as np

words_array = np.array(words_array)

In [34]:
np.linalg.norm(words_array[1, :] - words_array[2, :])

23.122328

In [32]:
words_array[1, :]

array([-2.759961,  1.117283, -0.107931, -0.152149, -2.290422, -0.172126,
        4.05315 , -1.791256, -1.659794,  0.4952  , -2.806745, -2.546781,
       -0.571918,  2.483599, -1.154938, -1.144642, -1.997079,  2.778277,
       -4.63895 ,  3.812113, -2.464403, -0.661001,  1.018622, -8.009574,
        0.163577,  1.77995 ,  1.08292 ,  2.203077,  1.701301,  2.771147,
       -1.84412 , -3.020437,  5.353816,  1.048164,  2.296658, -0.811624,
        0.300312,  2.013064, -1.484625,  2.062446, -1.034696,  2.141481,
        3.774813, -2.611755,  1.802854,  2.16944 , -1.446178,  1.244814,
        2.497016, -0.842132, -3.307101, -3.557271,  1.294654, -0.25647 ,
        0.022632,  1.007145,  1.610226, -3.296503,  3.143573,  1.180332,
       -4.1695  ,  5.196706,  2.109448, -0.790676,  0.711407,  3.816041,
        0.636172, -3.780181,  2.575389, -4.240666, -3.64649 , -0.419408,
       -1.553977,  0.79523 , -2.492866, -0.30542 , -1.775612, -0.205165,
        0.837055,  3.297613, -0.948421,  0.43182 , 

### Klastrowanie za pomocą DBSCAN

In [73]:
word2vec.similarity("flanela", "flanelowy")

0.7080588

In [79]:
import sklearn
sklearn.metrics.pairwise_distances(X = word2vec["flanela"].reshape(1,-1), Y = word2vec["flanelowy"].reshape(1,-1), metric="cosine")

array([[0.29194117]], dtype=float32)

In [11]:
from sklearn.cluster import DBSCAN

In [117]:
clustering = DBSCAN(eps=0.25, min_samples = 10, metric = "cosine").fit(words_array)

In [133]:
print("Klastry: ")
print(set((clustering.labels_)))

Klastry: 
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, -1}


In [127]:
import collections

# Słownik w którym kluczami są słowa, a wartościami numer klastra w którym znajduje się to słowo
word2cluster = dict()

# Słownik w którym kluczem jest numer klastra, a wartością lista słów w klastrze
cluster2word = collections.defaultdict(list)

for idx, word in enumerate(included_words):
    word2cluster[word] = clustering.labels_[idx]
    cluster2word[clustering.labels_[idx]].append(word)

In [129]:
print(f"Number of outliers: {len(cluster2word[-1])}")
for i in range(len(cluster2word) - 1):
    print(f"cluster {i}: {len(cluster2word[i])}")

Number of outliers: 6412
cluster 0: 770
cluster 1: 882
cluster 2: 746
cluster 3: 95
cluster 4: 113
cluster 5: 22
cluster 6: 19
cluster 7: 11
cluster 8: 26
cluster 9: 10
cluster 10: 42
cluster 11: 11
cluster 12: 15
cluster 13: 9
cluster 14: 7


Problem jest taki, że niektóre klastry są bardzo małe. Z drugiej strony niektóre z nich wydaja się bardzo sensowne:

In [134]:
def print_cluster(cluster):
    words_long_string = ""
    for word in cluster:
        words_long_string += word
        words_long_string += ", "
    print(words_long_string)

In [136]:
print("Klaster samochodów:")
print_cluster(cluster2word[3])

Klaster samochodów
cadillac, hatchback, saab, citroen, motocykl, gti, willys, kombi, scuderia, wrangler, aventador, opel, sedan, mazda, peugeot, mercedes, favorit, meriva, gt, ciągnik, autobus, narty, unimog, dodge, arrinera, tiguan, hulajnoga, traktor, volkswagen, sanie, ambulans, bmw, cruiser, nissan, porsche, pojazd, aygo, volvo, roomster, sanki, cars, przyczepa, seat, terenówka, yaris, helikopter, transporter, metalic, samochód, karoq, rower, jaguar, konie, rover, passat, garaż, kangoo, ciężarówka, lamborghini, ferrari, wóz, fiat, hummer, wagon, corolla, skuter, lotus, wózek, śmieciarka, sportage, audi, speedster, bugatti, combi, aut, vario, maserati, kabriolet, trabant, chevrolet, touran, toyota, vw, renault, jeep, charger, ford, auto, samolot, touareg, kodiaq, rowerek, stonic, buick, suv, 


In [138]:
print("Klaster włoskich słów:")
print_cluster(cluster2word[4])

Klaster włoskich słów:
balenciaga, giulia, fiori, arianna, iglesias, colle, raffaele, verona, enrico, dimas, matera, emporio, morales, alves, revlon, arnaldo, gucci, cattaneo, salerno, palermo, salinas, rinascimento, lopez, lecco, caserta, milano, escada, bustamante, dkny, fabia, carbone, flavia, trapani, felice, suarez, acqua, pezzo, narciso, pomar, diavolo, rivera, ricci, bari, habana, lagerfeld, prato, prisma, modena, fano, gilmar, enrique, chiavari, vallejo, americano, puig, ragusa, bianco, federica, mario, roberto, estrella, ronaldo, herrera, moschino, catania, versace, cacharel, armani, missoni, giorgio, biagiotti, carlo, siena, sibilla, cavalli, toscana, cristiano, gabbana, parma, diamanti, fratelli, andria, rojo, dino, rodriguez, renato, carrara, brito, cerruti, prada, arturo, savona, diego, luca, chanel, novara, ruiz, salvatore, ignacio, francesco, gio, antonio, asfora, trussardi, camil, escalante, valeria, capello, gino, conca, rossi, tronto, pedro, 


In [140]:
print("Klaster Amerykańskich miast:")
print_cluster(cluster2word[5])

Klaster Amerykańskich miast:
denver, wichita, wisconsin, peoria, kansas, milwaukee, dallas, vermont, rockford, cedar, tucson, detroit, fresno, chicago, georgia, kalifornia, michigan, boston, minnesota, seattle, iowa, hartford, 


In [142]:
print("Klaster określeń kształtu:")
print_cluster(cluster2word[6])

Klaster określeń kształtu:
owalny, naroże, sześciokątny, okrągły, trójkątny, poprzeczny, prostokąt, zakrzywiony, podłużny, schodkowy, skośny, stożkowy, prostokątny, kolumnowy, kwadratowy, półokrągły, cylindryczny, kwadrat, płaski, 


In [147]:
print("Klaster (zdrobnionych) zwierząt:")
print_cluster(cluster2word[7])

Klaster (zdrobnionych) zwierząt:
kociak, pieski, zwierzak, niedźwiadek, bobas, kaczuszka, kota, zwierzątko, piesek, małpka, maluch, 


In [148]:
print("Klaster zwierząt:")
print_cluster(cluster2word[8])

Klaster zwierząt:
pies, wilk, królik, hipopotam, kaczka, słoń, kot, troll, nosorożec, smok, leniwiec, tygrys, mysz, zając, żółw, lampart, niedźwiedź, jeleń, małpa, wiewiórka, myszy, brontozaur, krokodyl, wąż, pantera, żyrafa, 


In [150]:
print("Klaster (łacińskich?) słów:")
print_cluster(cluster2word[9])

Klaster (łacińskich?) słów:
movit, velit, editi, quid, postquam, vinum, magno, grati, legere, aio, 


In [152]:
print("Klaster francuskich słów:")
print_cluster(cluster2word[10])

Klaster francuskich słów:
clinique, un, univers, chefs, bien, professionnel, champagne, poudre, visionnaire, que, ses, tous, cuisine, nouvel, parfum, chaud, reve, fantome, curso, fils, roue, boxe, classique, ou, petit, mais, vertus, soleil, etoile, adorable, espanol, oiseau, elle, parisienne, absolue, plume, noms, jaune, homme, jeune, lumineuse, c'est, 


In [156]:
print("Klaster zbitek liter:")
print_cluster(cluster2word[11])

Klaster zbitek liter:
pj, bi, jt, jb, jj, pbd, ia, q, aq, xq, uz, 


In [158]:
print("Klaster biżuterii:")
print_cluster(cluster2word[12])

Klaster biżuterii:
zapinka, bransoletka, kolczyk, amulet, breloczek, bransoletki, paciorki, koraliki, perła, klips, diadem, pierścionek, naszyjnik, brelok, wisiorek, 


In [160]:
print("Klaster imion:")
print_cluster(cluster2word[13])

Klaster imion:
emil, gustav, heiko, sven, oskar, fischer, anton, klaus, otto, 


In [161]:
print("Klaster hiszpańskich słów:")
print_cluster(cluster2word[14])

Klaster określeń kształtu:
vida, viento, sono, paloma, soy, agua, siempre, 
